<a href="https://colab.research.google.com/github/dr-mushtaq/Chatbot-in-e-learning-system/blob/main/Model_Training_Mushtaq_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**Table of Content**</p>



1.   Import Library
2.   Import Dataset
3.   Data Preprocessing
4.   Neural Network




#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**1-Import library**</p>

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**2-Import Dataset**</p>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/gdrive/MyDrive/Research /Research Projects/Chatbot System in Education /Dataset/intents.json').read()
intents = json.loads(data_file)


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**3-Data Preprocessing**</p>

In [5]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
 import nltk
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)



235 documents
93 classes [' ADMISSION OF FOREIGN/DUAL NATIONAL', ' Additional Specialization Certificate', ' Admission Status', ' Alumni Association ', ' Citizen complaint portal ', ' Club and Societies ', ' Courses Catalogue', ' Degree/Transcript Verification ', ' Digiskills ', ' Disabled Student Fee structure(Local)', ' Disabled Student Fee structure(Overseas)', ' Exam Superintendent Registration ', ' FACULTY OF SCIENCE & TECHNOLOGY Programs', ' Faculty of Arts Programs', ' Faculty of Education Programs', ' Faculty of Management Programs', ' Grading Scheme', ' How to Apply Online', ' Life at VU', ' New Registration', ' ORIC ', ' PROFESSIONAL COURSES DEVELOPMENT', ' PROTECTION AGAINST HARASSMENT  ', ' Prospectus', ' RIGHT OF ACCESS TO INFORMATION ', ' Recent Advertisements of VU', ' Schedule of other Charges', ' Short Certificate Course', ' Student Startup', ' Study Scheme', ' Suitability of  VU', ' Tender', ' Vendor Registration ', ' Zero Semester', 'Academic Calendar', 'Acceptable a

In [9]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

**Create our training data**

In [10]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)

# Padding to ensure consistent shape for bag-of-words representations
max_len = max(len(sample[0]) for sample in training)  # Find the maximum length
padded_training = []
for sample in training:
    bag, output_row = sample
    # Check if padding is actually necessary
    if len(bag) < max_len:
        padded_bag = bag + [0] * (max_len - len(bag))  # Pad with zeros if needed
    else:
        padded_bag = bag  # No padding needed
    padded_training.append([padded_bag, output_row])

training = np.array(padded_training, dtype=object) # Convert the padded list to NumPy array, allow object dtype


# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


**Suggested Optimization**

1. Padding is unnecessary for bag-of-words

Bag-of-words (BoW) representations are already of fixed length because the word list (words) defines the dimensionality. The for w in words loop ensures each vector has the same length. Thus, the explicit padding step is redundant.

2. Potential Issue with np.array Conversion

np.array(training, dtype=object) is used, but NumPy arrays are better suited for numerical data. Consider using np.array(training, dtype=np.float32) if all values are numerical.

3. Shuffling Before NumPy Conversion

You correctly shuffle the data before converting it to a NumPy array, which is good practice to ensure randomness in training.

In [11]:
import random
import numpy as np

# Create our training data
training = []

# Create an empty array for output (one-hot encoded)
output_empty = [0] * len(classes)

# Training set: bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = [1 if w in [lemmatizer.lemmatize(word.lower()) for word in doc[0]] else 0 for w in words]

    # Output is '1' for current tag, '0' for others
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle and convert to NumPy array
random.shuffle(training)
training = np.array(training, dtype=object)

# Split features and labels
train_x = list(training[:, 0])  # Input patterns
train_y = list(training[:, 1])  # Corresponding intents

print("Training data created")


Training data created


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4-Model Training**</p>

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.1-Neural Netowrk**</p>


##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.1.1 Model Training**</p>

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Changed 'lr' to 'learning_rate'
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

**Observations & Suggestions**

1. **Dropout Rate**

You are using 0.5 dropout after both hidden layers, which is quite aggressive. This might slow down training or cause underfitting. Try 0.3 instead of 0.5.

2- **Optimizer Choice**

SGD with Nesterov momentum works well, but Adam optimizer (optimizer='adam') is often better for deep learning models in NLP tasks.

If you want to stick with SGD, consider learning rate decay scheduling to improve training convergence.

3. **Epochs & Batch Size Tuning**

epochs=200 might be excessive unless you have a large dataset. Try 100 epochs first, then increase if needed.

batch_size=5 is quite small. Try 16 or 32 for better efficiency.

4. **Convert Lists to NumPy Arrays Before Training**

Ensure train_x and train_y are explicitly converted to NumPy arrays before feeding them into model.fit().



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np

# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.3))  # Reduced dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Reduced dropout
model.add(Dense(len(train_y[0]), activation='softmax'))

# Choose optimizer (SGD with decay or Adam)
optimizer = Adam(learning_rate=0.001)  # Adam usually performs better
# optimizer = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)  # Optional

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

# Train the model
hist = model.fit(train_x, train_y, epochs=100, batch_size=16, verbose=1)

# Save the trained model
model.save("chatbot_model.h5")

print("Model training complete and saved as chatbot_model.h5")

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.0168 - loss: 4.5451
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0250 - loss: 4.4858    
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0350 - loss: 4.4431     
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0869 - loss: 4.3896 
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1504 - loss: 4.3196 
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1643 - loss: 4.2314 
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1344 - loss: 4.1026 
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1796 - loss: 3.9737 
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1873 - loss: 3.8201 
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2587 - loss: 3.5627 
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2342 - loss: 3.5088 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3

Model training complete and saved as chatbot_model.h5


##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.1.2 Model Saving**</p>

In [ ]:
model.save('chat_model.keras', hist) # Changed 'chat_model' to 'chat_model.keras' to include the .keras extension
print("model created")

model created



#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**5.1-Random Forest**</p>

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

# Convert lists to NumPy arrays
train_x = np.array(train_x)

# Convert one-hot labels to categorical labels (integer encoding)
label_encoder = LabelEncoder()
train_y_labels = label_encoder.fit_transform([np.argmax(y) for y in train_y])  # Convert one-hot to integer

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y_labels, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=500, max_depth=None, random_state=42, class_weight="balanced")
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Improved Random Forest Model Accuracy: {accuracy:.2f}")

# Save the model and label encoder
joblib.dump(rf_model, "chatbot_rf_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Random Forest model training complete and saved as chatbot_rf_model.pkl")


✅ Improved Random Forest Model Accuracy: 0.53
Random Forest model training complete and saved as chatbot_rf_model.pkl


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2- LSTM**</p>

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional

##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.1 - Model Architecture 1**</p>


Stacked LSTM (multi-layer LSTM) with Batch Normalization

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay # Import ExponentialDecay
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional

#Create the LSTM network
model = Sequential()
model.add(LSTM(64, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay( # Now you can use ExponentialDecay
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.01 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=128, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 290, 64)             │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 290, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 290, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 290, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 290, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 290, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 93)                  │           1,581 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,477 (134.68 KB)

 Trainable params: 34,253 (133.80 KB)

 Non-trainable params: 224 (896.00 B)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.0137 - loss: 4.6464
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.0054 - loss: 4.7265
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0054 - loss: 4.7115
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0028 - loss: 4.8089    
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0166 - loss: 4.6691
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0057 - loss: 4.6717    
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0355 - loss: 4.6880
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0057 - loss: 4.6615    
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0137 - loss: 4.7121
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0057 - loss: 4.6758    
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0083 - loss: 4.6417
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accur

##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.2 - Model Architecture 2**</p>

Deeper Stacked LSTM architecture

In [ ]:
#Create the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 290, 128)            │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 290, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 290, 64)             │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 290, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 290, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 290, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 290, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 290, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 93)                  │           1,581 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,549 (521.68 KB)

 Trainable params: 133,325 (520.80 KB)

 Non-trainable params: 224 (896.00 B)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.0140 - loss: 4.6680
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.0220 - loss: 4.6230
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.0111 - loss: 4.7041
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0166 - loss: 4.6874
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0166 - loss: 4.7398
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0109 - loss: 4.6744
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0000e+00 - loss: 4.6417
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0054 - loss: 4.6869
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0054 - loss: 4.6950
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.0000e+00 - loss: 4.6158
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.0054 - loss: 4.6806
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.

##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.3 - Model Architecture 3**</p>


Bidirectional LSTMs

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 290, 128)            │          33,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 290, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 290, 64)             │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 290, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 290, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 32)                  │          10,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 93)                  │           3,069 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 88,829 (346.99 KB)

 Trainable params: 88,637 (346.24 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Epoch 1/100
932/932 [==============================] - 89s 80ms/step - loss: 4.5237 - accuracy: 0.0520
Epoch 2/100
932/932 [==============================] - 73s 79ms/step - loss: 3.7492 - accuracy: 0.1474
Epoch 3/100
932/932 [==============================] - 74s 79ms/step - loss: 3.2352 - accuracy: 0.2316
Epoch 4/100
932/932 [==============================] - 73s 79ms/step - loss: 2.7851 - accuracy: 0.3036
Epoch 5/100
932/932 [==============================] - 73s 79ms/step - loss: 2.4090 - accuracy: 0.3725
Epoch 6/100
932/932 [==============================] - 73s 78ms/step - loss: 2.0772 - accuracy: 0.4478
Epoch 7/100
932/932 [==============================] - 74s 79ms/step - loss: 1.7939 - accuracy: 0.5148
Epoch 8/100
932/932 [==============================] - 74s 80ms/step - loss: 1.5713 - accuracy: 0.5678
Epoch 9/100
932/932 [==============================] - 74s 80ms/step - loss: 1.3666 - accuracy: 0.6160
Epoch 10/100
932/932 [==============================] - 74s 80ms/step - l

In [ ]:
model.save( path_to_save_model+'chat_model_3', hist)
print("model created and saved")

model created and saved


##<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.4 - Model Architecture 4**</p>


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_7 (Bidirectional)      │ (None, 290, 256)            │         133,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 290, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_8 (Bidirectional)      │ (None, 290, 128)            │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 290, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 290, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 290, 64)             │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 290, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 290, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 32)                  │          10,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 93)                  │           3,069 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 353,021 (1.35 MB)

 Trainable params: 352,573 (1.34 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 136ms/step - accuracy: 0.0166 - loss: 4.8668
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.0083 - loss: 4.8738
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.0166 - loss: 4.8061
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.0054 - loss: 4.7760
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.0000e+00 - loss: 4.6855
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0194 - loss: 4.8023
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0111 - loss: 4.7610
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0113 - loss: 4.7219
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.0000e+00 - loss: 4.7324
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.0083 - loss: 4.7716
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0054 - loss: 4.7846
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - a

In [ ]:
model.save( path_to_save_model+'chat_model_4', hist)
print("model created and saved")

model created and saved


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 256)          133120    
 al)                                                             
                                                                 
 batch_normalization (Batch  (None, 391, 256)          1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 128)          164352    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 391, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 391, 64)           41216     
 onal)                                                  

In [ ]:
model.save( path_to_save_model+'chat_model_5', hist)
print("model created and saved")


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**BERT**</p>

In [1]:
import json
import pandas as pd

# Load intents.json
with open('/content/gdrive/MyDrive/Research /Research Projects/Chatbot System in Education /Dataset/intents.json') as f:
    data = json.load(f)

# Convert to DataFrame
dataset = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        dataset.append([pattern, intent['tag']])

df = pd.DataFrame(dataset, columns=['text', 'label'])

# Save as CSV
df.to_csv('intents.csv', index=False)
print("Dataset saved successfully!")


Dataset saved successfully!


In [2]:
pip install torch transformers datasets scikit-learn


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder

# Load dataset
dataset = load_dataset('csv', data_files='intents.csv')

# Extract the training split (since load_dataset returns a DatasetDict with a 'train' key)
dataset = dataset['train']

# Encode labels correctly
label_encoder = LabelEncoder()
dataset = dataset.map(lambda x: {'label': label_encoder.fit_transform([x['label']])[0]},
                      remove_columns=['label'])

# Ensure dataset is a DatasetDict after splitting
dataset = dataset.train_test_split(test_size=0.2)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization and remove text column
dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))

# Training Arguments
training_args = TrainingArguments(
    output_dir="./bert-intent-model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Check dataset keys
print("Dataset keys:", dataset.keys())  # Should print: dict_keys(['train', 'test'])

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

# Train model
trainer.train()



Map:   0%|          | 0/235 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Dataset keys: dict_keys(['train', 'test'])


...

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:


Abort: 

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**References**</p>

[Chatgpt Roadmap](https://chatgpt.com/c/67dd0d37-384c-800e-8fb3-e12a2efcaab1)